In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import os
import psycopg2
from config import *

In [2]:
# Connect to the database
conn = psycopg2.connect(
    host="ec2-54-147-36-107.compute-1.amazonaws.com",
    database="d1i9m18bh0nee",
    user="nrdsxccgistivu",
    password=pg_password)

# Read the data from the database
df1 = pd.read_sql_query('SELECT * FROM "eminem_comments"',con=conn)
df2 = pd.read_sql_query('SELECT * FROM "psy_comments"',con=conn)
df3 = pd.read_sql_query('SELECT * FROM "katyperry_comments"',con=conn)
df4 = pd.read_sql_query('SELECT * FROM "lmfao_comments"',con=conn)
df5 = pd.read_sql_query('SELECT * FROM "shakira_comments"',con=conn)

In [3]:
# concatenate the dataframes
df = pd.concat([df1, df2, df3, df4, df5])

df

,id,content,class
0,0,lovely girl talk xxx,1
1,1,always end coming back song br,0
2,2,sister received new rel nofollow class ot hash...,1
3,3,cool,0
4,4,hello palastine,1
...,...,...,...
362,365,love song sing camp time,0
363,366,love song two reasons africa born beautiful so...,0
364,367,wow,0
365,368,shakira u wiredo,0


In [21]:
len(df[df['class'] == 1])/len(df)

0.5047568710359408

In [4]:
# drop the comment_id, date, and video columns
# df = df.drop(['comment_id', 'author', 'date'], axis=1)

# convert the class column to integer
# df['class'] = df['class'].astype(int)

In [23]:
# Use machine learning to create a model that can predict whether a comment is spam or not spam

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['class'], random_state=42)

# Use CountVectorizer to create document-term matrices from X_train and X_test
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# Use Naive Bayes to predict the star rating for X_test_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# Calculate the accuracy of your model
from sklearn import metrics
accuracy_score = metrics.accuracy_score(y_test, y_pred_class)
print(accuracy_score)

# Calculate the AUC
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
auc_score = metrics.roc_auc_score(y_test, y_pred_prob)
print(auc_score)

print('-------------------------------------')

# Print the confusion matrix
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_class)
print(confusion_matrix)

print('-------------------------------------')

# Print the classification report
print(metrics.classification_report(y_test, y_pred_class))



0.9069767441860465
0.9625089605734767
-------------------------------------
[[197  28]
 [ 16 232]]
-------------------------------------
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       225
           1       0.89      0.94      0.91       248

    accuracy                           0.91       473
   macro avg       0.91      0.91      0.91       473
weighted avg       0.91      0.91      0.91       473



In [24]:
# import the what happened csv
df_new = pd.read_csv('Clean_Data/Cleaned_Files/clean_Snarky_Puppy.csv')

df_new

,Unnamed: 0,text
0,0,funky keyboard riff solo part without baseline...
1,1,crazy well two drum sets play together clash
2,2,fell like god probably snarky puppy play list ...
3,3,NaN
4,4,tracks visually registered like track find han...
...,...,...
1297,1297,back roots live album recording today feel luc...
1298,1298,nice
1299,1299,expecting go hans zimmer happy
1300,1300,watching yall show kc yall killing lol


In [25]:
# drop id author thumbsup # of replies and replies columns
# df_new = df_new.drop(['id', 'author', 'thumbsup', '# of replies', 'replies'], axis=1)

In [26]:
df_new

,Unnamed: 0,text
0,0,funky keyboard riff solo part without baseline...
1,1,crazy well two drum sets play together clash
2,2,fell like god probably snarky puppy play list ...
3,3,NaN
4,4,tracks visually registered like track find han...
...,...,...
1297,1297,back roots live album recording today feel luc...
1298,1298,nice
1299,1299,expecting go hans zimmer happy
1300,1300,watching yall show kc yall killing lol


In [27]:
# drop na
df_new = df_new.dropna()

In [28]:
# use the model to predict the class of the new comments
X_new_dtm = vect.transform(df_new['text'])
new_pred_class = nb.predict(X_new_dtm)

# add the new class to the dataframe
df_new['class'] = new_pred_class

df_new

c:\Users\macdk\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,text,class
0,0,funky keyboard riff solo part without baseline...,0
1,1,crazy well two drum sets play together clash,0
2,2,fell like god probably snarky puppy play list ...,0
4,4,tracks visually registered like track find han...,1
5,5,jack conte see mark,1
...,...,...,...
1297,1297,back roots live album recording today feel luc...,1
1298,1298,nice,0
1299,1299,expecting go hans zimmer happy,0
1300,1300,watching yall show kc yall killing lol,1


In [29]:
# % of comments marked spam
len(df_new[df_new['class'] == 1])/len(df_new)

0.2891183478951549

In [12]:
# use SVM to predict the class of the new comments
from sklearn.svm import SVC
svm = SVC(probability=True)
svm.fit(X_train_dtm, y_train)
svm_pred_class = svm.predict(X_new_dtm)

# add the new class to the dataframe
df_new['class'] = svm_pred_class

df_new['class'].value_counts()

c:\Users\macdk\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    1174
1      85
Name: class, dtype: int64

In [13]:
df_new

,Unnamed: 0,text,class
0,0,funky keyboard riff solo part without baseline...,0
1,1,crazy well two drum sets play together clash,0
2,2,fell like god probably snarky puppy play list ...,0
4,4,tracks visually registered like track find han...,0
5,5,jack conte see mark,0
...,...,...,...
1297,1297,back roots live album recording today feel luc...,0
1298,1298,nice,0
1299,1299,expecting go hans zimmer happy,0
1300,1300,watching yall show kc yall killing lol,0


In [22]:
len(df_new[df_new['class'] == 1])/len(df_new)

0.06751389992057188